In [1]:
import pandas as pd
import requests
import bs4
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gfmfb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gfmfb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
wiki = "https://en.wikipedia.org/wiki/List_of_pop_musicians_who_died_of_drug_overdose"
page = requests.get(wiki)
soup = BeautifulSoup(page.content,'html.parser')
names = set()
tables = soup.findAll(name='table', class_='wikitable sortable')


for table in tables:
    body = table.findChild("tbody" , recursive=False)
    for row in body.findAll('tr'):
        cells = row.findAll('a')
        if(len(cells) == 0) :continue
        names.add(cells[0].text.strip())

In [3]:
names

{'Alan Wilson',
 'Allen Woody',
 'Amy Winehouse',
 'Andrew Wood',
 'Billy Mackenzie',
 'Billy Murcia',
 'Bobby Hatfield',
 'Bobby Sheehan',
 'Bon Scott',
 'Bradley Nowell',
 'Brent Mydland',
 'Brian Cole',
 'Carl Crack',
 'Chris Cornell',
 'Chris Kelly',
 'Danny Whitten',
 'Darby Crash',
 'Dave Brockie',
 'Dave Rubinstein',
 'Dave Schulthise',
 'David McComb',
 'David Ruffin',
 'Dee Dee Ramone',
 'Dickie Pride',
 'Dinah Washington',
 'Dwayne Goettel',
 'Erik Brødreskift',
 'Esther Phillips',
 'Frankie Lymon',
 'GG Allin',
 'Gary Holton',
 'Gary Moore',
 'Gary Thain',
 'Gerry Rafferty',
 'Gidget Gein',
 'Gram Parsons',
 'Gregory Herbert',
 'Hillel Slovak',
 'Hollywood Fats',
 'Howie Epstein',
 'Ike Turner',
 'Iosu Expósito',
 'James Honeyman-Scott',
 'Jani Lane',
 'Janis Joplin',
 'Jason Molina',
 'Jay Bennett',
 'Jay Reatard',
 'Jeremy Michael Ward',
 'Jesse Ed Davis',
 'Jim Morrison',
 'Jimi Hendrix',
 'Jimi Jamison',
 'Jimmy McCulloch',
 'John Baker Saunders',
 'John Belushi',
 'John

In [4]:
len(names)

112

In [5]:
page = requests.get("https://drugabuse.com/30-famous-musicians-who-have-battled-drug-addiction-and-alcoholism/")
soup = BeautifulSoup(page.content,'html.parser')
for entry in soup.findAll("h2"):
    try:
        names.add(entry.text.split(".",1)[1].strip())
    except:
        break
    

In [6]:
print(len(names))
namesPD = pd.DataFrame(data={"Band": list(names)})

138


In [7]:
songsTable1 = pd.read_csv("lyrics/lyrics1.csv")

In [8]:
songsTable1.head()

,Band,Lyrics,Song
0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday
1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die
2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside
3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot
4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds


In [9]:
pd1= songsTable1.merge(namesPD, how="inner", on="Band")
pd2= songsTable1[songsTable1['Band'].isin(names)]

In [10]:
print(len(pd1))
print(len(pd2))


#This is just to show that these are the the same frames, that the two methods in the above cells do that same thing
pd1.sort_values(by=["Band",'Lyrics','Song'],inplace=True)
pd2.sort_values(by=["Band",'Lyrics','Song'],inplace=True)

pd2.reset_index(drop=True,inplace=True)
pd1.reset_index(drop=True,inplace=True)

print(pd2.equals(pd1))

2673
2673
True


C:\Users\gfmfb\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [11]:
pd2[pd2.duplicated(keep=False)]
pd1[pd1.duplicated(keep=False)]

#there are duplicates, need to drop later

,Band,Lyrics,Song
1989,Ray Charles,For all we know\r\nWe may never meet again\r\n...,For All We Know
1990,Ray Charles,For all we know\r\nWe may never meet again\r\n...,For All We Know


In [12]:
songsTable2 = pd.read_csv("lyrics/lyrics2.csv")
pd3= songsTable2.merge(namesPD, how="inner", on="Band")
pd4= songsTable2[songsTable2['Band'].isin(names)]
pd3.sort_values(by=["Band",'Lyrics','Song'],inplace=True)
pd4.sort_values(by=["Band",'Lyrics','Song'],inplace=True)

pd3 = pd3.reset_index(drop=True)
pd4 = pd4.reset_index(drop=True)

print(pd3.equals(pd4))
pd3[pd3.duplicated(keep=False)]
pd4[pd4.duplicated(keep=False)]

True


C:\Users\gfmfb\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Band,Lyrics,Song


In [13]:
len(pd3)


4036

In [14]:
len(pd4)

4036

In [15]:
pd3.head()

,Band,Lyrics,Song
0,Amy Winehouse,All I can ever be to you is a darkness that we...,Tears Dry on Their Own [Clean Version]
1,Amy Winehouse,All I can ever be to you is a darkness that we...,Tears Dry on Their Own [DVD]
2,Amy Winehouse,All I can ever be to you is a darkness that we...,Tears Dry on Their Own
3,Amy Winehouse,"Attracts me, till it hurts to concentrate,\r\n...",Amy Amy Amy
4,Amy Winehouse,"Attracts me, till it hurts to concentrate,\r\n...",Amy Amy Amy (Outro)


In [16]:
pd2.head()

,Band,Lyrics,Song
0,Amy Winehouse,"All I can ever be to you,\r\nIs a darkness tha...",Tears Dry [Original Version]
1,Amy Winehouse,Build your dreams to the stars above\r\nBut wh...,Don't Go To Strangers [Multimedia Track]
2,Amy Winehouse,I can't wait to get away from you\r\nUnsurpris...,"Best Friends, Right?"
3,Amy Winehouse,I can't wait to get away from you\r\nUnsurpris...,"Best Friends, Right? [Leicester Summer Sundae ..."
4,Amy Winehouse,I would die before I divorce you\r\nI'd take a...,Between the Cheats


In [17]:
pd5 = pd.concat([pd1,pd2])

In [18]:
pd5

,Band,Lyrics,Song
0,Amy Winehouse,"All I can ever be to you,\r\nIs a darkness tha...",Tears Dry [Original Version]
1,Amy Winehouse,Build your dreams to the stars above\r\nBut wh...,Don't Go To Strangers [Multimedia Track]
2,Amy Winehouse,I can't wait to get away from you\r\nUnsurpris...,"Best Friends, Right?"
3,Amy Winehouse,I can't wait to get away from you\r\nUnsurpris...,"Best Friends, Right? [Leicester Summer Sundae ..."
4,Amy Winehouse,I would die before I divorce you\r\nI'd take a...,Between the Cheats
5,Amy Winehouse,I've been so many places in my life and time\r...,A Song for You
6,Amy Winehouse,Lent you outsiders and my new Badu\r\nWhile yo...,You Sent Me Flying [Demo] [Original Demo]
7,Amy Winehouse,"My heart is sad and lonely\nFor you I sigh, fo...",Body & Soul
8,Amy Winehouse,Nobody stands in between me and my man\r\nIt's...,Me & Mr Jones [Multimedia Track]
9,Amy Winehouse,Our day will come (our day will come)\r\nAnd w...,Our Day Will Come


In [19]:
#Cleaning the remixes
def getRidOfBrackets(input):
    return re.sub("[\(\[].*?[\)\]]", "", input).strip()
    
    
pd5["Song"] = pd5["Song"].apply(getRidOfBrackets)
pd5 = pd5.drop_duplicates(subset='Song', keep='first')

In [20]:
pd5

,Band,Lyrics,Song
0,Amy Winehouse,"All I can ever be to you,\r\nIs a darkness tha...",Tears Dry
1,Amy Winehouse,Build your dreams to the stars above\r\nBut wh...,Don't Go To Strangers
2,Amy Winehouse,I can't wait to get away from you\r\nUnsurpris...,"Best Friends, Right?"
4,Amy Winehouse,I would die before I divorce you\r\nI'd take a...,Between the Cheats
5,Amy Winehouse,I've been so many places in my life and time\r...,A Song for You
6,Amy Winehouse,Lent you outsiders and my new Badu\r\nWhile yo...,You Sent Me Flying
7,Amy Winehouse,"My heart is sad and lonely\nFor you I sigh, fo...",Body & Soul
8,Amy Winehouse,Nobody stands in between me and my man\r\nIt's...,Me & Mr Jones
9,Amy Winehouse,Our day will come (our day will come)\r\nAnd w...,Our Day Will Come
10,Amy Winehouse,Simple sweet guitar\r\nHumbled by the bass\r\n...,Halftime


In [21]:
stop = stopwords.words('english')

In [22]:
pd5['Lyrics'] = pd5['Lyrics'].str.lower()
pd5['Lyrics'] = pd5['Lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

C:\Users\gfmfb\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\gfmfb\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
pd5

,Band,Lyrics,Song
0,Amy Winehouse,"ever you, darkness knew, regret get accustomed...",Tears Dry
1,Amy Winehouse,build dreams stars need someone love go strang...,Don't Go To Strangers
2,Amy Winehouse,can't wait get away unsurprisingly hate commun...,"Best Friends, Right?"
4,Amy Winehouse,would die divorce i'd take thousand thumps lov...,Between the Cheats
5,Amy Winehouse,i've many places life time i've sung lot songs...,A Song for You
6,Amy Winehouse,lent outsiders new badu thinking clue tough so...,You Sent Me Flying
7,Amy Winehouse,"heart sad lonely sigh, dear seen i'm you, body...",Body & Soul
8,Amy Winehouse,nobody stands man mr. jones (me mr. jones) kin...,Me & Mr Jones
9,Amy Winehouse,day come (our day come) we'll everything we'll...,Our Day Will Come
10,Amy Winehouse,simple sweet guitar humbled bass beat kicks ev...,Halftime
